# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [34]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np


In [35]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [36]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [37]:
group = data.groupby(["CustomerID","ProductName"], as_index=True).agg({"Quantity":sum})
group.head()
 

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [39]:
  
matrix=pd.pivot_table(group, values='Quantity', index=['ProductName'], columns=['CustomerID'],aggfunc=np.sum).fillna(0)
matrix.head()

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0    0.0    0.0   
Apricots - Dried                     1.0    0.0    0.0    0.0    1.0    0.0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube            0.0    1.0    0.0    0.0  ...    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0  ...   25.0   
Appetizer - Mushroom Tart            0.0    1.0    0.0    0.0  ...   25.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0  ...    0.0   
Apricots - Dried                     0.0    0.0    0.0    0.0  ...    0.0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube           25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp   25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0   25.0   25.0   25.0   
Apricots - Dried                    25.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube            0.0    0.0    0.0  
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0  
Appetizer - Mushroom Tart            0.0   25.0    0.0  
Appetizer - Sausage Rolls            0.0   25.0    0.0  
Apricots - Dried                     0.0    0.0    0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [42]:
distances = pdist(matrix.T, 'euclidean')
print(distances)
distances_matrix = squareform(distances)
display(distances_matrix)

distances = pd.DataFrame(1/(1 + distances_matrix), index=matrix.columns, columns=matrix.columns)
display(distances)

[ 11.91637529  10.48808848  11.22497216 ... 304.13812651 305.16389039
 303.10889132]


array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.077421  0.087047  0.081800  0.080634  0.082709   
200         0.077421  1.000000  0.078448  0.076435  0.073693  0.075255   
264         0.087047  0.078448  1.000000  0.080070  0.081800  0.080350   
356         0.081800  0.076435  0.080070  1.000000  0.076435  0.078187   
412         0.080634  0.073693  0.081800  0.076435  1.000000  0.078711   
...              ...       ...       ...       ...       ...       ...   
97928       0.004515  0.004521  0.004543  0.004526  0.004530  0.004533   
98069       0.004583  0.004614  0.004595  0.004578  0.004578  0.004578   
98159       0.004355  0.004367  0.004365  0.004365  0.004367  0.004363   
98185       0.004167  0.004166  0.004179  0.004175  0.004177  0.004167   
98200       0.004333  0.004335  0.004333  0.004339  0.004349  0.004347   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.074573  0.083020  0.081503  0.080070  ...  0.004811  0.004669   
200         0.075956  0.076435  0.077674  0.076923  ...  0.004824  0.004681   
264         0.076923  0.080634  0.082100  0.078448  ...  0.004822  0.004674   
356         0.075025  0.082403  0.077171  0.075956  ...  0.004816  0.004671   
412         0.075025  0.082403  0.078187  0.078448  ...  0.004810  0.004702   
...              ...       ...       ...       ...  ...       ...       ...   
97928       0.004534  0.004526  0.004528  0.004523  ...  0.003509  0.003509   
98069       0.004589  0.004581  0.004585  0.004592  ...  0.003509  0.003509   
98159       0.004366  0.004363  0.004369  0.004365  ...  0.003509  0.003565   
98185       0.004168  0.004176  0.004172  0.004161  ...  0.003311  0.003162   
98200       0.004338  0.004335  0.004329  0.004347  ...  0.003418  0.003345   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004412  0.005019  0.004312  0.004515  0.004583  0.004355   
200         0.004431  0.005047  0.004311  0.004521  0.004614  0.004367   
264         0.004416  0.005035  0.004322  0.004543  0.004595  0.004365   
356         0.004416  0.005038  0.004310  0.004526  0.004578  0.004365   
412         0.004414  0.005034  0.004318  0.004530  0.004578  0.004367   
...              ...       ...       ...       ...       ...       ...   
97928       0.003300  0.003653  0.003579  1.000000  0.003638  0.003418   
98069       0.003369  0.003509  0.003496  0.003638  1.000000  0.003469   
98159       0.003322  0.003431  0.003322  0.003418  0.003469  1.000000   
98185       0.003255  0.003406  0.003172  0.003255  0.003345  0.003277   
98200       0.003381  0.003444  0.003288  0.003245  0.003381  0.003266   

CustomerID     98185     98200  
CustomerID                      
33          0.004167  0.004333  
200         0.004166  0.004335  
264         0.004179  0.004333  
356         0.004175  0.004339  
412         0.004177  0.004349  
...              ...       ...  
97928       0.003255  0.003245  
98069       0.003345  0.003381  
98159       0.003277  0.003266  
98185       1.000000  0.003288  
98200       0.003288  1.000000  

[1000 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [61]:
customers = distances[33]
customers


CustomerID
33       1.000000
200      0.077421
264      0.087047
356      0.081800
412      0.080634
           ...   
97928    0.004515
98069    0.004583
98159    0.004355
98185    0.004167
98200    0.004333
Name: 33, Length: 1000, dtype: float64

In [68]:
result = list(customers.sort_values(ascending = False).index)[1:6]
result

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [76]:
custom = group.loc[result]
custom.head()

Quantity
CustomerID ProductName                         
264        Apricots - Halves                  1
           Apricots Fresh                     1
           Bacardi Breezer - Tropical         1
           Bagel - Plain                      1
           Banana - Leaves                    1

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [95]:
prod = custom.groupby(["ProductName"], as_index=True).agg({"Quantity":sum})
Prodquan = prod.sort_values('Quantity', ascending=False)
Prodquan.head()

Quantity
ProductName                            
Butter - Unsalted                     3
Wine - Ej Gallo Sierra Valley         3
Towels - Paper / Kraft                3
Soup - Campbells Bean Medley          3
Wine - Blue Nun Qualitatswein         3

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [98]:
produ = pd.merge(matrix, Prodquan, on = "ProductName").sort_values(by="Quantity",ascending=False)
display(produ)
l5 = list(produ[produ[33]==0][33].index)[:5]
l5

33  200  264  356  412  464  477  639  649  \
ProductName                                                                  
Butter - Unsalted              0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
Wine - Ej Gallo Sierra Valley  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0   
Towels - Paper / Kraft         1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0   
Soup - Campbells Bean Medley   0.0  0.0  1.0  0.0  0.0  0.0  2.0  0.0  0.0   
Wine - Blue Nun Qualitatswein  0.0  0.0  1.0  1.0  0.0  0.0  0.0  0.0  0.0   
...                            ...  ...  ...  ...  ...  ...  ...  ...  ...   
Hinge W Undercut               0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0   
Ice Cream Bar - Hageen Daz To  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
Jagermeister                   0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
Jolt Cola - Electric Blue      0.0  0.0  0.0  1.0  1.0  0.0  0.0  1.0  0.0   
Yogurt - French Vanilla        1.0  0.0  0.0  1.0  0.0  0.0  2.0  0.0  0.0   

                               669  ...  97753  97769  97793  97900  97928  \
ProductName                         ...                                      
Butter - Unsalted              0.0  ...    0.0    0.0    0.0    0.0    0.0   
Wine - Ej Gallo Sierra Valley  0.0  ...    0.0   25.0   50.0    0.0   25.0   
Towels - Paper / Kraft         0.0  ...    0.0    0.0    0.0    0.0    0.0   
Soup - Campbells Bean Medley   0.0  ...    0.0    0.0   25.0   50.0   25.0   
Wine - Blue Nun Qualitatswein  0.0  ...    0.0    0.0    0.0    0.0    0.0   
...                            ...  ...    ...    ...    ...    ...    ...   
Hinge W Undercut               0.0  ...    0.0    0.0   25.0   25.0    0.0   
Ice Cream Bar - Hageen Daz To  0.0  ...    0.0    0.0    0.0    0.0    0.0   
Jagermeister                   0.0  ...    0.0    0.0    0.0    0.0    0.0   
Jolt Cola - Electric Blue      0.0  ...    0.0    0.0    0.0    0.0    0.0   
Yogurt - French Vanilla        1.0  ...    0.0   25.0    0.0    0.0    0.0   

                               98069  98159  98185  98200  Quantity  
ProductName                                                          
Butter - Unsalted                0.0    0.0   25.0    0.0         3  
Wine - Ej Gallo Sierra Valley    0.0    0.0    0.0    0.0         3  
Towels - Paper / Kraft           0.0    0.0    0.0    0.0         3  
Soup - Campbells Bean Medley     0.0    0.0    0.0    0.0         3  
Wine - Blue Nun Qualitatswein    0.0    0.0   25.0    0.0         3  
...                              ...    ...    ...    ...       ...  
Hinge W Undercut                 0.0    0.0    0.0    0.0         1  
Ice Cream Bar - Hageen Daz To   25.0    0.0    0.0    0.0         1  
Jagermeister                     0.0    0.0   25.0    0.0         1  
Jolt Cola - Electric Blue        0.0    0.0    0.0    0.0         1  
Yogurt - French Vanilla          0.0    0.0    0.0   25.0         1  

[218 rows x 1001 columns]

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [103]:
recommendations = {"CustomerID":[],"Product_1":[],"Product_2":[],"Product_3":[],"Product_4":[],"Product_5":[]}
customers = list(distances.keys())

for customer in customers:
    recommendations["CustomerID"].append(customer)
    results = list(distances[customer].sort_values(ascending=False).index[1:6])
    similar_customers = group.loc[results]
    ranked_products = similar_customers.groupby(["ProductName"],as_index=True).agg({"Quantity":sum}).sort_values("Quantity",ascending=False)
    top_5 = pd.merge(matrix, ranked_products, on="ProductName").sort_values(by="Quantity",ascending=False)
    list_5 = list(top_5[top_5[customer]==0][customer].index)[:5]
    recommendations["Product_1"].append(list_5[0])
    recommendations["Product_2"].append(list_5[1])
    recommendations["Product_3"].append(list_5[2])
    recommendations["Product_4"].append(list_5[3])
    recommendations["Product_5"].append(list_5[4])



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [105]:
result = pd.DataFrame(recommendations)
result.head()

CustomerID                     Product_1                        Product_2  \
0          33             Butter - Unsalted    Wine - Ej Gallo Sierra Valley   
1         200  Soup - Campbells Bean Medley  Muffin - Carrot Individual Wrap   
2         264       Soupfoamcont12oz 112con       Wine - Two Oceans Cabernet   
3         356         Veal - Inside, Choice                    Lamb - Ground   
4         412       Olive - Spread Tapenade      Sprouts - Baby Pea Tendrils   

                         Product_3                      Product_4  \
0     Soup - Campbells Bean Medley  Wine - Blue Nun Qualitatswein   
1                         Bay Leaf                  Pork - Kidney   
2  Bread - Italian Roll With Herbs          Veal - Inside, Choice   
3                          Pomello              Butter - Unsalted   
4    Wine - Blue Nun Qualitatswein          Pepper - Black, Whole   

                      Product_5  
0           Chicken - Soup Base  
1                   Wanton Wrap  
2  Fish - Scallops, Cold Smoked  
3        Tart Shells - Sweet, 4  
4  Soup - Campbells Bean Medley

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.